<a href="https://colab.research.google.com/github/tomochan001/hyappon/blob/master/ans_preprocess_knock_R_Colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# データサイエンス100本ノック（構造化データ加工編） - R

## はじめに

- 初めに以下のセルを実行してください
- 必要なライブラリのインポートと~~データベース（PostgreSQL）からのデータ読み込みを行います~~
- 利用が想定されるライブラリは以下セルでインポートしています
- その他利用したいライブラリがあれば適宜インストールしてください（!マークに続けてOSコマンドを入力することで、任意のubuntu Linuxコマンドが入力可能）
- 名前、住所等はダミーデータであり、実在するものではありません

In [ ]:
require('tidyr')
require('dplyr')
require('stringr')
require('caret')
require('lubridate')
require('rsample')
require('unbalanced')
require('readr')

Loading required package: caret

Warning message in library(package, lib.loc = lib.loc, character.only = TRUE, logical.return = TRUE, :
“there is no package called ‘caret’”
Loading required package: rsample

Warning message in library(package, lib.loc = lib.loc, character.only = TRUE, logical.return = TRUE, :
“there is no package called ‘rsample’”
Loading required package: unbalanced

Warning message in library(package, lib.loc = lib.loc, character.only = TRUE, logical.return = TRUE, :
“there is no package called ‘unbalanced’”
Loading required package: readr



In [4]:
# データを github/he-Japan-DataScientist-Society/100knocks-preprocess/master/docker/work/data/ フォルダよりDataframe形式でRead
df_customer = read.csv('https://raw.githubusercontent.com/The-Japan-DataScientist-Society/100knocks-preprocess/master/docker/work/data/customer.csv', quote="", stringsAsFactors=F)
df_category = read.csv('https://raw.githubusercontent.com/The-Japan-DataScientist-Society/100knocks-preprocess/master/docker/work/data/category.csv', quote="", stringsAsFactors=F)
df_product = read.csv('https://raw.githubusercontent.com/The-Japan-DataScientist-Society/100knocks-preprocess/master/docker/work/data/product.csv', quote="", stringsAsFactors=F)
df_receipt = read.csv('https://raw.githubusercontent.com/The-Japan-DataScientist-Society/100knocks-preprocess/master/docker/work/data/receipt.csv', quote="", stringsAsFactors=F)
df_store = read.csv('https://raw.githubusercontent.com/The-Japan-DataScientist-Society/100knocks-preprocess/master/docker/work/data/store.csv', quote="", stringsAsFactors=F)
# df_geocode = read.csv('https://raw.githubusercontent.com/noguhiro2002/100knocks-preprocess_ForColab-AzureNotebook/master/data/geocode.csv', quote="", stringsAsFactors=F)
df_geocode = read.csv('https://github.com/The-Japan-DataScientist-Society/100knocks-preprocess/blob/master/docker/work/data/geocode.csv', quote="", stringsAsFactors=F) # エラーが出る

In [5]:
str(df_receipt)

'data.frame':	104681 obs. of  9 variables:
 $ sales_ymd     : int  20181103 20181118 20170712 20190205 20180821 20190605 20181205 20190922 20170504 20191010 ...
 $ sales_epoch   : int  1257206400 1258502400 1215820800 1265328000 1250812800 1275696000 1259971200 1285113600 1209859200 1286668800 ...
 $ store_cd      : chr  "S14006" "S13008" "S14028" "S14042" ...
 $ receipt_no    : int  112 1132 1102 1132 1102 1112 1102 1102 1112 1102 ...
 $ receipt_sub_no: int  1 2 1 1 2 1 2 1 2 1 ...
 $ customer_id   : chr  "CS006214000001" "CS008415000097" "CS028414000014" "ZZ000000000000" ...
 $ product_cd    : chr  "P070305012" "P070701017" "P060101005" "P050301001" ...
 $ quantity      : int  1 1 1 1 1 1 1 1 1 1 ...
 $ amount        : int  158 81 170 25 90 138 30 128 770 680 ...


>R-001: レシート明細データフレーム（df_receipt）から全項目の先頭10件を表示し、どのようなデータを保有しているか目視で確認せよ。

In [6]:
head(df_receipt, n = 10)

,sales_ymd,sales_epoch,store_cd,receipt_no,receipt_sub_no,customer_id,product_cd,quantity,amount
,<int>,<int>,<chr>,<int>,<int>,<chr>,<chr>,<int>,<int>
1,20181103,1257206400,S14006,112,1,CS006214000001,P070305012,1,158
2,20181118,1258502400,S13008,1132,2,CS008415000097,P070701017,1,81
3,20170712,1215820800,S14028,1102,1,CS028414000014,P060101005,1,170
4,20190205,1265328000,S14042,1132,1,ZZ000000000000,P050301001,1,25
5,20180821,1250812800,S14025,1102,2,CS025415000050,P060102007,1,90
6,20190605,1275696000,S13003,1112,1,CS003515000195,P050102002,1,138
7,20181205,1259971200,S14024,1102,2,CS024514000042,P080101005,1,30
8,20190922,1285113600,S14040,1102,1,CS040415000178,P070501004,1,128
9,20170504,1209859200,S13020,1112,2,ZZ000000000000,P071302010,1,770


>R-002: レシート明細データフレーム（df_receipt）から売上日（sales_ymd）、顧客ID（customer_id）、商品コード（product_cd）、売上金額（amount）の順に列を指定し、10件表示させよ。

In [8]:
head(df_receipt[c("sales_ymd", "customer_id", "product_cd", "amount")], n=10)

,sales_ymd,customer_id,product_cd,amount
,<int>,<chr>,<chr>,<int>
1,20181103,CS006214000001,P070305012,158
2,20181118,CS008415000097,P070701017,81
3,20170712,CS028414000014,P060101005,170
4,20190205,ZZ000000000000,P050301001,25
5,20180821,CS025415000050,P060102007,90
6,20190605,CS003515000195,P050102002,138
7,20181205,CS024514000042,P080101005,30
8,20190922,CS040415000178,P070501004,128
9,20170504,ZZ000000000000,P071302010,770
